In [1]:
import numpy as np
import matplotlib.pyplot as plt
from create_orbital_values import *
from datetime import datetime as dt

In [2]:
def fac(n):
    return 1 if n <= 1 else n * fac(n - 1)

def binom(i, j):
    if i > j:
        return 0
    return int(fac(j) / (fac(i) * fac(j - i)))

mat = np.fromfunction(np.vectorize(binom), shape=(5, 5)).astype(np.float32)
a_exp_tens = np.fromfunction(np.vectorize(lambda i, j: max(0, j - i)), shape=(5, 5))

In [3]:
class Gaussian:
    def __init__(self, R, alpha, coeff_vector, mat=mat, a_exp_tens=a_exp_tens):
        self.R = R
        self.alpha = alpha
        self.coeff_vector = coeff_vector
        self.mat = mat
        self.a_exp_tens = a_exp_tens

    @staticmethod
    def convolve_tensors(tens1, tens2):
        conv = np.zeros_like(tens1)
        d = 5
        for i in range(d):
            for j in range(d):
                for k in range(d):
                    conv[i, j, k] = (tens1[i::-1, j::-1, k::-1] * tens2[:i+1, :j+1, :k+1]).sum()
        return conv

    @staticmethod
    def shift_poly(vec, a, mat, a_exp_tens):
        tens = a ** a_exp_tens
        mat *= tens
        res = np.einsum("ij,j->i", mat, vec)
        return res

    @staticmethod
    def mul(gauss1, gauss2):
        t1 = dt.now()
        R1 = gauss1.R
        alpha1 = gauss1.alpha
        sigma1 = 0.5 / alpha1
        coeff_vector1 = gauss1.coeff_vector
        for i in range(3):
            coeff_vector1 = np.apply_along_axis(Gaussian.shift_poly, axis=i, arr=coeff_vector1, a=-R1[i], mat=gauss1.mat, a_exp_tens=gauss1.a_exp_tens)

        t2 = dt.now()
        R2 = gauss2.R
        alpha2 = gauss2.alpha
        sigma2 = 0.5 / alpha2
        coeff_vector2 = gauss2.coeff_vector
        for i in range(3):
            coeff_vector1 = np.apply_along_axis(Gaussian.shift_poly, axis=i, arr=coeff_vector1, a=-R2[i], mat=gauss2.mat, a_exp_tens=gauss2.a_exp_tens)
        
        t3 = dt.now()
        coeff_vector_mult = Gaussian.convolve_tensors(coeff_vector1, coeff_vector2)
        t4 = dt.now()
        sigma_mult = sigma1 * sigma2 / (sigma1 + sigma2)
        R_mult = (R1 * sigma2 + R2 * sigma1) / (sigma1 + sigma2)
        for i in range(3):
            coeff_vector_mult = np.apply_along_axis(Gaussian.shift_poly, axis=i, arr=coeff_vector1, a=-R_mult[i], mat=gauss2.mat, a_exp_tens=gauss2.a_exp_tens)
        coeff_vector_mult *= np.exp(-np.linalg.norm(R1 - R2) ** 2 / (2 * sigma1 + sigma2))
        alpha_mult = 0.5 / sigma_mult
        res = Gaussian(R_mult, alpha_mult, coeff_vector_mult)
        t5 = dt.now()
        print(f"d1: {t2 - t1}")
        print(f"d2: {t3 - t2}")
        print(f"d3: {t4 - t3}")
        print(f"d4: {t5 - t4}")
        return res
    
    def integration(self):
        int_value = np.array([
            np.sqrt(np.pi / self.alpha),
            0,
            np.sqrt(np.pi) / (2 * self.alpha ** 1.5),
            0,
            3 * np.sqrt(np.pi) / (4 * self.alpha ** 2.5)
        ])
        int_values = int_value[:, None, None] * int_value[None, :, None] * int_value[None, None, :]
        int_values *= self.coeff_vector
        int_values = int_values.sum()
        return int_values

In [4]:
Z = np.array([7]) # atom = N
coeffs_orig, exponents = create_coeffs_tensor_retry()
coeffs_orig = coeffs_orig[Z - 1][:, None]
exponents = exponents[Z - 1][:, None]
prefactors = create_prefactors(exponents)
coeffs = coeffs_orig * prefactors
print(coeffs[0, 0, 3])
norms_eval = norms(exponents, coeffs)
a = coeffs.copy()
coeffs /= norms_eval 
coeffs = coeffs[0, 0, :]
exponents = exponents[0, 0, :]

[-1.4792578  -1.1800719  -0.54803591 -0.         -0.         -0.
 -0.         -0.         -0.        ]
[-1.4792578  -1.1800719  -0.54803591 -0.         -0.         -0.
 -0.         -0.         -0.        ]


In [5]:
coeff_vector_1 = np.fromfunction(lambda i, j, k: (i == 0) * (j == 0) * (k == 0), shape=(5, 5, 5)).astype(np.float32)
coeff_vector_x = np.fromfunction(lambda i, j, k: (i == 1) * (j == 0) * (k == 0), shape=(5, 5, 5)).astype(np.float32)
coeff_vector_y = np.fromfunction(lambda i, j, k: (i == 0) * (j == 1) * (k == 0), shape=(5, 5, 5)).astype(np.float32)
coeff_vector_z = np.fromfunction(lambda i, j, k: (i == 0) * (j == 0) * (k == 1), shape=(5, 5, 5)).astype(np.float32)
coeff_vector_xx = np.fromfunction(lambda i, j, k: (i == 2) * (j == 0) * (k == 0), shape=(5, 5, 5)).astype(np.float32)
coeff_vector_yy = np.fromfunction(lambda i, j, k: (i == 0) * (j == 2) * (k == 0), shape=(5, 5, 5)).astype(np.float32)
coeff_vector_zz = np.fromfunction(lambda i, j, k: (i == 0) * (j == 0) * (k == 2), shape=(5, 5, 5)).astype(np.float32)

coeff_vector = np.stack([
    coeff_vector_1,
    coeff_vector_1,
    coeff_vector_1,
    coeff_vector_x,
    coeff_vector_y,
    coeff_vector_z,
    coeff_vector_x,
    coeff_vector_y,
    coeff_vector_z,
    Gaussian.convolve_tensors(coeff_vector_x, coeff_vector_y), #xy
    Gaussian.convolve_tensors(coeff_vector_z, coeff_vector_y), #yz
    2 * coeff_vector_zz - coeff_vector_xx - coeff_vector_yy,   # 3z^2 - r^2
    Gaussian.convolve_tensors(coeff_vector_z, coeff_vector_y), #xz
    coeff_vector_xx - coeff_vector_yy
])

exponents_vector = exponents
exponents_vector.shape

(14, 9)

In [6]:
gaussian_tens = np.fromfunction(np.vectorize(lambda i, j: Gaussian(np.zeros(3,), exponents_vector[i, j], coeff_vector[i])), shape=((14, 9)), dtype=np.int32)

In [7]:
res = np.fromfunction(np.vectorize(lambda i, j, k, l: Gaussian.mul(gaussian_tens[i, k], gaussian_tens[j, l])), dtype=np.int32, shape=(14, 14, 9, 9)) 

d1: 0:00:00.000788
d2: 0:00:00.000777
d3: 0:00:00.000343
d4: 0:00:00.000484
d1: 0:00:00.000504
d2: 0:00:00.000446
d3: 0:00:00.000339
d4: 0:00:00.000791
d1: 0:00:00.000527
d2: 0:00:00.000632
d3: 0:00:00.000335
d4: 0:00:00.000469
d1: 0:00:00.000438
d2: 0:00:00.000427
d3: 0:00:00.000325
d4: 0:00:00.000458
d1: 0:00:00.000445
d2: 0:00:00.000536
d3: 0:00:00.000339
d4: 0:00:00.000530
d1: 0:00:00.000479
d2: 0:00:00.000465
d3: 0:00:00.000392
d4: 0:00:00.000614
d1: 0:00:00.000339
d2: 0:00:00.000339
d3: 0:00:00.000322
d4: 0:00:00.000350
d1: 0:00:00.000332
d2: 0:00:00.000346
d3: 0:00:00.000305
d4: 0:00:00.000346
d1: 0:00:00.000340
d2: 0:00:00.000331
d3: 0:00:00.000310
d4: 0:00:00.000355
d1: 0:00:00.000338
d2: 0:00:00.000331
d3: 0:00:00.000304
d4: 0:00:00.000354
d1: 0:00:00.000337
d2: 0:00:00.000329
d3: 0:00:00.000305
d4: 0:00:00.000357
d1: 0:00:00.000335
d2: 0:00:00.000332
d3: 0:00:00.000308
d4: 0:00:00.000354
d1: 0:00:00.000338
d2: 0:00:00.000333
d3: 0:00:00.000304
d4: 0:00:00.000355
d1: 0:00:00.